**Hierarchical Indexing** 

Up to this point we've been focusing primarily on one-dimensional and two-dimensional data, stored with Pandas Series and DataFrame objects. Pandas does proved Panel and Panel4D objects which handle three-dimensional and four-dimensional data. The more common practice is to make use of Hierarchical indexing, to use multiple index levels.

This section will focuse on MultiIndex objects, consideration around indexing, slicing, computing statistics across multiply indexed data, and routines for converting between simple and hierarchical indexed representations of data

In [14]:
import pandas as pd
import numpy as np


*A Multiply Indexed Series*

Let's start by considering how we might represent two-dimensional data within a one-dimensional Series.

The bad way

If you would like to track data about states from two different years, using Pandas tools talked about already you might use a tuple as keys:

In [15]:
index = [('California', 2000), ('California', 2010), ('New York', 2000), ('New York', 2010), ('Texas', 2000), ('Texas', 2010)]
populations = [33871648, 37253956, 18976457, 19378102, 20851820, 25145561]
pop = pd.Series(populations, index=index)
pop

(California, 2000)    33871648
(California, 2010)    37253956
(New York, 2000)      18976457
(New York, 2010)      19378102
(Texas, 2000)         20851820
(Texas, 2010)         25145561
dtype: int64

You can index or slice the series based on this multiple index:

In [16]:
pop[('California', 2010):('Texas', 2000)]

(California, 2010)    37253956
(New York, 2000)      18976457
(New York, 2010)      19378102
(Texas, 2000)         20851820
dtype: int64

But to only get values from 2010 it might get messy:

In [17]:
pop[[i for i in pop.index if i[1]==2010]]

(California, 2010)    37253956
(New York, 2010)      19378102
(Texas, 2010)         25145561
dtype: int64

That works but it's not very clean or effecient for large data bases

**Pandas MultiIndex**

We can do it with a multi index from the tuples:

In [18]:
index = pd.MultiIndex.from_tuples(index)
index

MultiIndex([('California', 2000),
            ('California', 2010),
            (  'New York', 2000),
            (  'New York', 2010),
            (     'Texas', 2000),
            (     'Texas', 2010)],
           )

In [19]:
pop = pop.reindex(index)
pop

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

Here the first two column show the multiple index values while the third shows the data. Blank entries represent the same value as the line above it

We can access all data for which the second index is 2010 using pandas slice notation:

In [20]:
pop[:, 2010]

California    37253956
New York      19378102
Texas         25145561
dtype: int64

**MultiIndex as extra dimension**

We can store the same data using a DataFrame, Pandas has the unstack() method that will quickly convert a multiply-indexed Series into a indexed DataFrame

In [21]:
pop_df = pop.unstack()
pop_df

,2000,2010
California,33871648,37253956
New York,18976457,19378102
Texas,20851820,25145561


With *stack()* providing the opposite operation:

In [22]:
pop_df.stack()

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

You are able to use multi-indexing to represent two-dimensional data within a one-dimensional Series. Each extra level in a multi-indexing represents an extra dimension of data we can represent. It is easy to add another column of data with a MultiIndex

In [23]:
pop_df = pd.DataFrame({'total':pop, 'under 18': [9267089, 9284094, 4687374, 4318033, 5906301, 6879014]})
pop_df

total  under 18
California 2000  33871648   9267089
           2010  37253956   9284094
New York   2000  18976457   4687374
           2010  19378102   4318033
Texas      2000  20851820   5906301
           2010  25145561   6879014

In [24]:
# all ufuncs work 
# Compute the fraction of people under 18 by given year
f_u18 = pop_df['under 18'] / pop_df['total']
f_u18.unstack()

,2000,2010
California,0.273594,0.249211
New York,0.247010,0.222831
Texas,0.283251,0.273568


**Methods of MultiIndex Creation**

The most straightforward way to construct a multiply indexed Series or DataFrame is to pass a list of two or more indexed arrays to the constructor. For example:

In [25]:
df = pd.DataFrame(np.random.rand(4, 2), index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]], columns=['data1', 'data2'])
df

data1     data2
a 1  0.362427  0.598119
  2  0.531910  0.942869
b 1  0.888019  0.845277
  2  0.909156  0.150298

If you pass a dictionary with tuples as keys, Pandas will automatically recognize this and use a MultiIndex by default:

In [26]:
data = {('California', 2000): 33871648,
('California', 2010): 37253956,
('Texas', 2000): 20851820,
('Texas', 2010): 25145561,
('New York', 2000): 18976457,
('New York', 2010): 19378102}
pd.Series(data)

California  2000    33871648
            2010    37253956
Texas       2000    20851820
            2010    25145561
New York    2000    18976457
            2010    19378102
dtype: int64

**Explicit MultiIndex constructors**

For more flexibility in how the index is constructed, you can instead use the class method constructors available in the pd.MultiIndex

In [27]:
pd.MultiIndex.from_arrays([['a', 'a', 'b', 'b'], [1, 2, 1, 2]])

MultiIndex([('a', 1),
            ('a', 2),
            ('b', 1),
            ('b', 2)],
           )

In [28]:
# From tuples
pd.MultiIndex.from_tuples([('a', 1), ('b', 1), ('a',2), ('b', 2)])

MultiIndex([('a', 1),
            ('b', 1),
            ('a', 2),
            ('b', 2)],
           )

In [29]:
# Single indeces
pd.MultiIndex.from_product([['a', 'b'], [1, 2]])

MultiIndex([('a', 1),
            ('a', 2),
            ('b', 1),
            ('b', 2)],
           )

In [38]:
pd.MultiIndex(levels=[['a', 'b'], [1, 2]], labels=[[0,0,1,1],[0, 1, 0, 1]])

TypeError: __new__() got an unexpected keyword argument 'labels'

MultiIndex level Names

Sometimes it's more convienient to name the levels of the MultiIndex. You can accomplish this by passing the names argument to any MultiIndex constructors.

In [41]:
pop.index.names=['state','year']
pop

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

MultIndex for columns

In DataFrame, the rows and columns are completely symmetric, rows can have multiple levels of indices, and so can columns.

In [52]:
#hierarchical indices and columns
index = pd.MultiIndex.from_product([[2013, 2014], [1, 2]], names=['year','visit'])
columns = pd.MultiIndex.from_product([['Bob', 'Guido', 'Sue'],['HR','Temp']],names=['patient','type'])

# Mock Data
data = np.round(np.random.randn(4, 6), 1)
data[:, ::2] *= 10
data += 37

# Create the DataFrame
health_data = pd.DataFrame(data, index=index, columns=columns)
health_data

patient      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      34.0  38.0  53.0  37.2  35.0  37.2
     2      38.0  37.3  36.0  36.8  30.0  37.9
2014 1      38.0  36.4  36.0  37.7  20.0  37.3
     2      39.0  36.3  50.0  35.6  50.0  38.8

In [53]:
health_data['Guido']

type          HR  Temp
year visit            
2013 1      53.0  37.2
     2      36.0  36.8
2014 1      36.0  37.7
     2      50.0  35.6

Indexing and Slicing a MultiIndex

We'll first look at indexing multiply indexed Series, and then multiply indexed DataFrames.

In [54]:
pop

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [55]:
pop['California', 2000]

33871648

In [56]:
pop['California']

year
2000    33871648
2010    37253956
dtype: int64

In [57]:
pop.loc['California':'New York']

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
dtype: int64

In [58]:
pop[:, 2000]

state
California    33871648
New York      18976457
Texas         20851820
dtype: int64

In [59]:
pop[pop > 22000000]

state       year
California  2000    33871648
            2010    37253956
Texas       2010    25145561
dtype: int64

In [60]:
pop[['California', 'Texas']]

state       year
California  2000    33871648
            2010    37253956
Texas       2000    20851820
            2010    25145561
dtype: int64

**Multiply indexed DataFrames**

A Multiply indexed DataFrame behaves in a similar manner. Consider our toy medical DataFrame from before:

In [61]:
health_data

patient      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      34.0  38.0  53.0  37.2  35.0  37.2
     2      38.0  37.3  36.0  36.8  30.0  37.9
2014 1      38.0  36.4  36.0  37.7  20.0  37.3
     2      39.0  36.3  50.0  35.6  50.0  38.8

Columns are a primary in a DataFrames, and the syntax used for multiply indexed Series applies to the columns. We can recover Guido's Heart Rate data:

In [62]:
health_data['Guido','HR']

year  visit
2013  1        53.0
      2        36.0
2014  1        36.0
      2        50.0
Name: (Guido, HR), dtype: float64

Can use the loc, iloc, and ix indexers

In [63]:
health_data.iloc[:2, :2]

patient      Bob      
type          HR  Temp
year visit            
2013 1      34.0  38.0
     2      38.0  37.3

These indexers provide an array-like view of the underlying two-dimensional data, but each individual index in loc or iloc can be passeda tuple of multiple indices.

In [64]:
health_data.loc[:, ('Bob', 'HR')]

year  visit
2013  1        34.0
      2        38.0
2014  1        38.0
      2        39.0
Name: (Bob, HR), dtype: float64

Trying to create a slice within a tuple will lead to syntax error:

In [65]:
health_data.loc[(:, 1), (:, 'HR')]

SyntaxError: invalid syntax (3311942670.py, line 1)

You could get around this by building the desired slice explicitly using Python's built in slice() function, but a better way is to use an IndexSlice object provided by Pandas:

In [66]:
idx = pd.IndexSlice
health_data.loc[idx[:, 1], idx[:, 'HR']]

,patient,Bob,Guido,Sue
,type,HR,HR,HR
year,visit,,,
2013,1,34.0,53.0,35.0
2014,1,38.0,36.0,20.0


**Rearranging Multi-Indices**

One of the keys to working with Mulitply indexed data is knowing how to effectively transform the data. 
There are operations that will preserve all the information in the dataset, but rearrange it for the purposes of computations (Ex: stack and unstack). There are many more ways to finely control the rearrangement of data between hierachical indices and columns:

***Sorted and unsorted indices***

Many of the MultiIndexing slicing operations will fail if the index is not sorted.

Start by creating some simply multiply indexed data where the indices are not lexographically sorted:

In [67]:
index = pd.MultiIndex.from_product([['a', 'c', 'b'], [1, 2]])
data = pd.Series(np.random.rand(6), index=index)
data.index.names = ['char', 'int']
data

char  int
a     1      0.986059
      2      0.066559
c     1      0.847428
      2      0.156155
b     1      0.296127
      2      0.315358
dtype: float64

Trying to partially slice this index will result in error:

In [68]:
try:
    data['a':'b']
except KeyError as e:
    print(type(e))
    print(e)

<class 'pandas.errors.UnsortedIndexError'>
'Key length (1) was greater than MultiIndex lexsort depth (0)'


This is the result of the MultiIndex not being sorted. Partial slices and other operations require the levels in the MultiIndex to be in sorted order. Pandas provides a number of convenience routines to perform this type of sorting; examples are sort_index(), sortlevel() methods of DataFrame:

In [69]:
data = data.sort_index()
data

char  int
a     1      0.986059
      2      0.066559
b     1      0.296127
      2      0.315358
c     1      0.847428
      2      0.156155
dtype: float64

In [70]:
data['a':'b']

char  int
a     1      0.986059
      2      0.066559
b     1      0.296127
      2      0.315358
dtype: float64

*Stacking and unstacking indices*

As we saw briefly before, it is possible to convert a dataset from a stacked multi-index to a simple two-dimensional representaion, optionally specifying the level to use:

In [71]:
pop.unstack(level=0)

state,California,New York,Texas
year,,,
2000,33871648,18976457,20851820
2010,37253956,19378102,25145561


In [72]:
pop.unstack(level=1)

year,2000,2010
state,,
California,33871648,37253956
New York,18976457,19378102
Texas,20851820,25145561


The opposite of unstack() is stack()

In [73]:
pop.unstack().stack()

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

*Index setting and resetting*

Another way to rearrang hierarchical data is to turn the index labels into columns; this can be done with the reset_index() method. Calling this on the populaiton dictionary will result in a DataFrame with a state and a year column. 

In [74]:
pop_flat = pop.reset_index(name='population')
pop_flat

,state,year,population
0,California,2000,33871648
1,California,2010,37253956
2,New York,2000,18976457
3,New York,2010,19378102
4,Texas,2000,20851820
5,Texas,2010,25145561


Often working with data in the real world, the raw data output looks like this and it's useful to build a MultiIndex from the column values. This can be done with the set_index method of the DataFrame, which returns a multiply indexed DataFrame:

In [75]:
pop_flat.set_index(['state', 'year'])

population
state      year            
California 2000    33871648
           2010    37253956
New York   2000    18976457
           2010    19378102
Texas      2000    20851820
           2010    25145561

*Data Aggregations on Multi-Indices*

We've previously seen that Pandas has built-in data aggregation methods, such as mean(), sum(), and max(). These can be passed as a level parameter that controls which subset of the data the aggregate is computed on.


In [76]:
health_data

patient      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      34.0  38.0  53.0  37.2  35.0  37.2
     2      38.0  37.3  36.0  36.8  30.0  37.9
2014 1      38.0  36.4  36.0  37.7  20.0  37.3
     2      39.0  36.3  50.0  35.6  50.0  38.8

To average out the measurements in the two visits each year. We can do this by naming the index level we'd like to explore, in this case the year:

In [77]:
data_mean = health_data.mean(level='year')
data_mean

/tmp/ipykernel_3523/2845460446.py:1: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.median(level=1) should use df.groupby(level=1).median().
  data_mean = health_data.mean(level='year')


patient   Bob        Guido          Sue       
type       HR   Temp    HR   Temp    HR   Temp
year                                          
2013     36.0  37.65  44.5  37.00  32.5  37.55
2014     38.5  36.35  43.0  36.65  35.0  38.05

By further making use of the axis keyword, we can take the mean among levels on the column as well:

In [78]:
data_mean.mean(axis=1, level='type')

/tmp/ipykernel_3523/1641729190.py:1: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.median(level=1) should use df.groupby(level=1).median().
  data_mean.mean(axis=1, level='type')


type,HR,Temp
year,,
2013,37.666667,37.400000
2014,38.833333,37.016667


In two lines we found the average heart rate and tempurature measured among all subjects in all visits in each year. This syntax is a shortcut to the GroupBy functionality